In [1]:
import pandas as pd
import os
import torch
import json
import numpy as np
from tqdm import tqdm
import pickle
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import subprocess
import numpy as np
import torch
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import os
from torch_geometric.loader import DataLoader
from pathlib import Path
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from collections import Counter
from torch_geometric.data import Data, Dataset
import json
import os
import pandas as pd
import pandas as pd
from rdkit import Chem
import torch
import numpy as np
from torch.utils.data import Dataset
import pickle
from collections import defaultdict
from pathlib import Path
import torch.nn.functional as F
import pickle

In [2]:
import matplotlib.pyplot as plt

In [3]:
%run ../Config.ipynb

In [4]:
config = Config()

/home/lbcb00/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


#### Set sent to be docked 

In [7]:
#refined_set_reduced = pd.read_csv('../Datahub/pdbs_refined_set.csv')

In [8]:
len(refined_set_reduced)

NameError: name 'refined_set_reduced' is not defined

In [ ]:
"4eor" in list(refined_set_reduced['PDB code'])

#### pdbs that were meant to be docked 

#### PDbs docked 

In [9]:
import os

def find_pdb_with_results(path, name):
    # Get the list of pdb folders in the specified path
    pdb_folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]

    # Initialize list to store pdb folders with 'results'
    pdb_with_results = []

    for pdb_folder in pdb_folders:
        pdb_path = os.path.join(path, pdb_folder)
        results_path = os.path.join(pdb_path, name)

        # Check if the 'results' folder exists in the pdb folder
        if os.path.exists(results_path) and os.path.isdir(results_path):
            pdb_with_results.append(pdb_folder)

    return pdb_with_results

# Specify the path to the 'set' folder
set_folder_path = '../coreset'

# Call the function to find pdb folders with 'results'
result_pdbs = find_pdb_with_results(set_folder_path, name = 'results')

#### QTD pdbs docked 

In [10]:
len(result_pdbs)

284

#### QTD Pdbs com rmsd calculdados com sucesso 

In [11]:
#rmsd = pd.read_csv(f"{config.data}/rmsd_refined_set.csv") # rmsd
rmsd = pd.read_csv(f"../rmsd_refined_set.csv") # rmsd

In [12]:
rmsd['pdb'].nunique()

3940

#### QTd pdbs com menos de 10 poses 

In [13]:
import seaborn as sns 

In [14]:
rmsd_pose_count = rmsd.groupby('pdb')['poserank'].nunique().sort_values().reset_index()

In [15]:
rmsd_pose_count[rmsd_pose_count['poserank'] !=10 ]['pdb'].nunique()

952

### Binana 

In [13]:
import os

def find_pdb_with_results(path, name):
    # Get the list of pdb folders in the specified path
    pdb_folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]

    # Initialize list to store pdb folders with 'results'
    pdb_with_results = []

    for pdb_folder in pdb_folders:
        pdb_path = os.path.join(path, pdb_folder)
        results_path = os.path.join(pdb_path, name, f'binana_{pdb_folder}_pose_1','log.txt')
        # Check if the 'results' folder exists in the pdb folder
        if os.path.exists(results_path) and os.path.isfile(results_path):
            pdb_with_results.append(pdb_folder)

    return pdb_with_results

# Specify the path to the 'set' folder
set_folder_path = '../coreset'

# Call the function to find pdb folders with 'results'
result_pdbs = find_pdb_with_results(set_folder_path, name = 'binana')

In [14]:
print("Pdbs processed by binana: ", len(result_pdbs))

Pdbs processed by binana:  284


#### QTD PDbs que o binana gerou resultados 

#### Tempo para rodar por quantidade de pdbs 

284 - PDBs - 29 min!

In [15]:
pdb = "5qa8"

In [16]:
pdb in list(refined_set_reduced['PDB code'])

NameError: name 'refined_set_reduced' is not defined

In [17]:
pdb in result_pdbs

False

#### General Report

| Metric                                      | Value |
|---------------------------------------------|-------|
| Set sent to be docked                       | 4790  |
| QTD pdbs docked                             | 3998  |
| QTD Pdbs com rmsd calculdados com sucesso   | 3940  |
| Pdbs processed by binana:                   | 3914  |
| Final available data to train (pose)            | 27651 |

In [21]:
class BipartiteData(Data):
    def __init__(self, edge_index=None, x_s=None, x_t=None, y=None, edge_attr=None, pdb = None, pose = None):
        super().__init__()
        self.edge_index = edge_index
        self.x_s = x_s
        self.x_t = x_t
        self.y = y
        self.edge_attr = edge_attr  # Add edge_attr attribute
        #self.num_nodes = len(set(edge_index[0].tolist())) +  len(set(edge_index[1].tolist()))
        self.num_nodes = (x_s.size(0) if x_s is not None else 0) + (x_t.size(0) if x_t is not None else 0)
        self.pdb = pdb
        self.pose = pose
    
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index':
            return torch.tensor([[self.x_s.size(0)], [self.x_t.size(0)]])
        else:
            return super().__inc__(key, value, *args, **kwargs)

In [35]:
cut = 4
name = f"bipartite_data_no_pose_rank_sorted_by_family_cut_{cut}_refined_set.pkl" 
name_encoded = name
# bipartite_data_with_pose_pred.pkl

In [36]:
with open(f'{config.data}/{name}', 'rb') as file:
    dataset_list_opened = pickle.load(file)
len(dataset_list_opened)

39140

In [38]:
from collections import Counter

In [44]:

print("Nº after created dataset",len(dataset_list_opened))
filtered_data_list_num_nodes = [data for data in dataset_list_opened if data.num_nodes > 0]
print("Nº after filtering based on number of nodes:", len(filtered_data_list_num_nodes))

filtered_data_list_descriptors = [data for data in filtered_data_list_num_nodes if data.x_s.shape[0] > 0 and data.x_t.shape[0] > 0]
print("Nº after filtering based on descriptor shapes:", len(filtered_data_list_descriptors))

filtered_data_list_descriptors = [data for data in filtered_data_list_descriptors if not data.x_t[0, 2].isnan()]
print("Nº after removing NaN values from descriptor:", len(filtered_data_list_descriptors))

filtered_data_list = filtered_data_list_descriptors.copy()

# Calculate label distribution
label_distribution = dict(Counter([label.y.tolist() for label in filtered_data_list]))
print("Label distribution:", label_distribution)

amount_of_graphs_used_to_train = len(filtered_data_list)
print("Total number of graphs used for training:", amount_of_graphs_used_to_train)

Nº after created dataset 39140
Nº after filtering based on number of nodes: 33167
Nº after filtering based on descriptor shapes: 27651
Nº after removing NaN values from descriptor: 27651
Label distribution: {0.0: 25442, 1.0: 2209}
Total number of graphs used for training: 27651


| Action                                      | Number |
|---------------------------------------------|--------|
| Created dataset                             | 39140  |
| Filtering based on number of nodes          | 33167  |
| Filtering based on descriptor shapes        | 27651  |
| Removing NaN values from descriptor         | 27651  |
| Label distribution                          | {0.0: 25442, 1.0: 2209} |
| Total number of graphs used for training    | 27651  |


In [46]:
rmsd = pd.read_csv(f"{config.data}/rmsd_refined_set.csv").sort_values(['pdb','poserank','RMSD']).drop_duplicates(['pdb','poserank'], keep = 'first')

sup = rmsd.groupby('pdb')[['poserank']].count().reset_index().rename(columns = {"poserank":"poserank_count"})
sup['unique_pdb_count'] = sup.groupby('poserank_count')['pdb'].transform('nunique')
sup = sup.drop(columns = ['pdb']).drop_duplicates().sort_values('poserank_count')

#### Final report 

#### General Report

| Metric                                      | Value |
|---------------------------------------------|-------|
| Set sent to be docked                       | 4790  |
| QTD pdbs docked                             | 3998  |
| QTD Pdbs com rmsd calculdados com sucesso   | 3940  |
| Pdbs processed by binana:                   | 3914  |
| Final available data to train (pose)            | 27651 |

| Action                                      | Number |
|---------------------------------------------|--------|
| Created dataset                             | 39140  |
| Filtering based on number of nodes          | 33167  |
| Filtering based on descriptor shapes        | 27651  |
| Removing NaN values from descriptor         | 27651  |
| Label distribution                          | {0.0: 25442, 1.0: 2209} |
| Total number of graphs used for training    | 27651  |

In [51]:
sup

,poserank_count,unique_pdb_count,qtd
107,1,75,75
95,2,83,166
156,3,60,180
7,4,50,200
33,5,60,300
67,6,61,366
212,7,70,490
1,8,91,728
10,9,402,3618
0,10,2988,29880


In [48]:
sup['qtd'] = sup['poserank_count']*sup['unique_pdb_count']

In [50]:
sup['qtd'].sum()

36003

In [52]:
27651* 0.2 

5530.200000000001

In [60]:
str(round(2209/(25442 + 2209)* 100, 2)) + " %" 

'7.99 %'

In [54]:
negative_instances = 25442
positive_instances = 2209
ratio = negative_instances / positive_instances
print(ratio)

11.517428700769578


In [61]:
dict(Counter([label.y.tolist() for label in filtered_data_list[22121:27650]]))

{0.0: 5092, 1.0: 437}

In [63]:
5092/437

11.652173913043478